In [723]:
# 메모리 변수 모두 제거
all = [var for var in globals() if var[0] != "_"]
for var in all:
    del globals()[var]

# 메모리 변수 확인
#globals()

In [724]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')
import pickle

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler 

from sklearn.metrics import mean_squared_error  # MSE RMSE
from sklearn.metrics import mean_absolute_error, r2_score # mae
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import eli5 
from eli5.sklearn import PermutationImportance 

In [725]:
df1 = pd.read_csv('data/train_메뉴제외.csv')
df3 = pd.read_csv('data/메뉴구분3.csv')

In [726]:
df_test_lunch = pd.read_csv('data/test_lunch.csv')
df_test_dinner = pd.read_csv('data/test_dinner.csv')

In [727]:
df_test_lunch.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '식사가능인원', '월', '중식강수여부', '중식기온', '중식강수량', '석식강수여부',
       '석식기온', '석식강수량', '확진자수', '휴일전날', '휴일다음날', '국분류', '메뉴분류'],
      dtype='object')

In [728]:
#plt.figure(figsize=(7,5))
#sns.heatmap(df1.corr())

## 스케일링
- y값과 카데고리 변수 제외하고 전부 스케일링

In [729]:
df_test_lunch.head(2)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,식사가능인원,월,중식강수여부,중식기온,중식강수량,석식강수여부,석식기온,석식강수량,확진자수,휴일전날,휴일다음날,국분류,메뉴분류
0,2021-01-27,수,2983,88,182,5,358.0,2355.0,1,0,8.4,0.0,0,7.6,0.0,559,0,0,탕,육류
1,2021-01-28,목,2983,104,212,409,348.0,2319.0,1,0,10.1,0.0,0,1.2,0.0,497,0,0,찌개,조류


### train_data

In [730]:
# 일자 드랍하고 y값 2개 뺴고 전부다 스케일링 해버리자
#df1['요일'] =  df1['요일'].astype('category') #요일을 카테고리화 하였다
#df1['요일'] = df1.요일.cat.codes #요일을 숫자로 바꿈
# 연, 월, 요일, 중식강수여부, 석시강수여부, 휴일전날, 휴일다음날 이 카데고리 전부 더미화 할지 생각해 보자
y = df1[['요일','중식계', '석식계','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날', '월']]

x = df1.drop(columns=['요일','중식계', '석식계','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날','월']) # 카테고리컬 변수와 y값으로 쓸 변수 일자변수 빼고 전부 스케일링
x = StandardScaler().fit_transform(x)

In [731]:
x = pd.DataFrame(x,columns=df1.drop(columns=['요일','중식계', '석식계','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날', '월']).columns)
df1 =pd.concat([x,y],axis=1)

### test_data_lunch

In [732]:
y = df_test_lunch[['요일','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날', '월', '국분류', '메뉴분류']]

x = df_test_lunch.drop(columns=['요일','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날', '월', '국분류', '메뉴분류'])
x = StandardScaler().fit_transform(x)
x = pd.DataFrame(x,columns=df_test_lunch.drop(columns=['요일','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날', '월', '국분류', '메뉴분류']).columns)
df_test_lunch =pd.concat([x,y],axis=1)

### test_data_dinner

In [733]:
y = df_test_dinner[['요일','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날', '월', '국분류', '메뉴분류']]

x = df_test_dinner.drop(columns=['요일','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날', '월', '국분류', '메뉴분류'])
x = StandardScaler().fit_transform(x)
x = pd.DataFrame(x,columns=df_test_dinner.drop(columns=['요일','일자','중식강수여부','석식강수여부','휴일전날', '휴일다음날', '월', '국분류', '메뉴분류']).columns)
df_test_dinner =pd.concat([x,y],axis=1)

In [734]:
#국분류 메뉴분류 때내기
df_test_lunch_cat = df_test_lunch[['국분류', '메뉴분류']]
df_test_dinner_cat = df_test_dinner[['국분류', '메뉴분류']]
df_test_lunch.drop(columns=['국분류', '메뉴분류'],inplace=True)
df_test_dinner.drop(columns=['국분류', '메뉴분류'],inplace=True)

In [735]:
df_test_lunch.columns 

Index(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '식사가능인원',
       '중식기온', '중식강수량', '석식기온', '석식강수량', '확진자수', '요일', '일자', '중식강수여부',
       '석식강수여부', '휴일전날', '휴일다음날', '월'],
      dtype='object')

In [736]:
df_test_dinner.columns

Index(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '식사가능인원',
       '중식기온', '중식강수량', '석식기온', '석식강수량', '확진자수', '요일', '일자', '중식강수여부',
       '석식강수여부', '휴일전날', '휴일다음날', '월'],
      dtype='object')

In [737]:
df1.columns

Index(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '식사가능인원',
       '중식결식', '석식결식', '중식기온', '중식강수량', '석식기온', '석식강수량', '확진자수', '요일', '중식계',
       '석식계', '일자', '중식강수여부', '석식강수여부', '휴일전날', '휴일다음날', '월'],
      dtype='object')

In [738]:
df1 =pd.concat([df1,df_test_lunch, df_test_dinner])


## df1 데이터 더미화

In [739]:
# 연, 월, 요일, 중식강수여부, 석시강수여부, 휴일전날, 휴일다음날 이 카데고리 전부 더미화 할지 생각해 보자
df1_cat = df1[['월', '요일']]


df1_cat = df1[['월', '요일']]
df1_cat['월'] =  df1_cat['월'].astype('category')
df1_cat = pd.get_dummies(df1_cat[['월', '요일']])# 더미화
df1.drop(columns=['월', '요일'],inplace=True)

df1 = pd.concat([df1,df1_cat],axis=1)

In [740]:
df_test_lunch = df1[-100:-50]
df_test_dinner = df1[-50:]
df1 = df1[:1205]

## 점심데이터

In [741]:
df3 = pd.concat([df3, df_test_lunch_cat, df_test_dinner_cat]) # -100:50 이 점심, -50 저녁

In [742]:
df_menu_cat=pd.get_dummies(df3[['국분류', '메뉴분류']])

In [743]:
df_lunch_cat = df3.iloc[:1205]
df_lunch_cat_dummy =pd.get_dummies(df_lunch_cat.iloc[:1205][['국분류', '메뉴분류']]) # 더미화

df_lunch = pd.concat([df1,df_lunch_cat_dummy],axis=1)

In [744]:
df_test_lunch = pd.concat([df_test_lunch,df_menu_cat[-100:-50]],axis=1)

In [745]:
#sns.heatmap(df_lunch.corr())

## 저녁데이터

In [746]:
df_dinner_cat_dummy =pd.get_dummies(df3.iloc[1205:-100][['국분류', '메뉴분류']]) # 더미화
df_dinner_cat_duumy = pd.concat([df3.iloc[1205:-100]['일자'],df_dinner_cat_dummy],axis=1) # 저녁값이 비어있는곳이 있어 merge 하기 위해 일자 값 추가

df_dinner =pd.merge(df1, df_dinner_cat_duumy,on='일자',how='left')

In [747]:
df_test_dinner = pd.concat([df_test_dinner,df_menu_cat[-50:]],axis=1)

In [748]:
df_dinner.isna().sum()

본사정원수              0
본사휴가자수             0
본사출장자수             0
본사시간외근무명령서승인건수     0
현본사소속재택근무자수        0
식사가능인원             0
중식결식               0
석식결식               0
중식기온               0
중식강수량              0
석식기온               0
석식강수량              0
확진자수               0
중식계                0
석식계                0
일자                 0
중식강수여부             0
석식강수여부             0
휴일전날               0
휴일다음날              0
월_1                0
월_2                0
월_3                0
월_4                0
월_5                0
월_6                0
월_7                0
월_8                0
월_9                0
월_10               0
월_11               0
월_12               0
요일_금               0
요일_목               0
요일_수               0
요일_월               0
요일_화               0
국분류_국             44
국분류_국수            44
국분류_스프            44
국분류_죽             44
국분류_찌개            44
국분류_탕             44
메뉴분류_기타           44
메뉴분류_육류           44
메뉴분류_조류           44
메뉴분류_해산물          44
dtype: int64

In [749]:
#df_dinner[df_dinner.국분류_국.isna()][['일자', '석식계']]

In [750]:
#df_dinner.fillna(0,inplace=True)
df_dinner.dropna(inplace=True) #저녁안먹는날 drop

In [751]:
df_dinner.isna().sum().sum()

0

In [752]:
#sns.heatmap(df_dinner.corr())

## 모델링 전 X,y 나누기

### 중식

In [753]:
df_test_lunch.columns

Index(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '식사가능인원',
       '중식결식', '석식결식', '중식기온', '중식강수량', '석식기온', '석식강수량', '확진자수', '중식계', '석식계',
       '일자', '중식강수여부', '석식강수여부', '휴일전날', '휴일다음날', '월_1', '월_2', '월_3', '월_4',
       '월_5', '월_6', '월_7', '월_8', '월_9', '월_10', '월_11', '월_12', '요일_금',
       '요일_목', '요일_수', '요일_월', '요일_화', '국분류_국', '국분류_국수', '국분류_스프', '국분류_우동',
       '국분류_죽', '국분류_찌개', '국분류_탕', '메뉴분류_기타', '메뉴분류_육류', '메뉴분류_조류',
       '메뉴분류_해산물'],
      dtype='object')

In [754]:
df_test_lunch = df_test_lunch[['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '식사가능인원',
       '중식기온', '중식강수량', '확진자수', '중식강수여부', '휴일전날', '휴일다음날', '월_1', '월_2', '월_3',
       '월_4', '월_5', '월_6', '월_7', '월_8', '월_9', '월_10', '월_11', '월_12',
       '요일_금', '요일_목', '요일_수', '요일_월', '요일_화', '국분류_국', '국분류_국수', '국분류_스프',
       '국분류_찌개', '국분류_탕', '메뉴분류_기타', '메뉴분류_육류', '메뉴분류_조류', '메뉴분류_해산물']]

In [755]:
X = df_lunch.drop(columns=['일자','중식계','석식계','석식결식','중식결식', '석식기온', '석식강수량', '석식강수여부'])

## 중식에 경우 ols summary에서 p_vale값이 중식기온이 유의미하지 않아 뻇다

#X = df_lunch[['요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수',
#       '식사가능인원', '중식강수여부', '중식기온', '중식강수량', '석식강수여부', '석식기온', '석식강수량', '확진자수',
#       '휴일전날', '휴일다음날']]
#X = df[['요일','본사정원수','본사휴가자수','본사출장자수','본사시간외근무명령서승인건수','현본사소속재택근무자수','식사가능인원']] #0.43
y = df_lunch['중식계']

In [756]:
X.columns

Index(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '식사가능인원',
       '중식기온', '중식강수량', '확진자수', '중식강수여부', '휴일전날', '휴일다음날', '월_1', '월_2', '월_3',
       '월_4', '월_5', '월_6', '월_7', '월_8', '월_9', '월_10', '월_11', '월_12',
       '요일_금', '요일_목', '요일_수', '요일_월', '요일_화', '국분류_국', '국분류_국수', '국분류_스프',
       '국분류_찌개', '국분류_탕', '메뉴분류_기타', '메뉴분류_육류', '메뉴분류_조류', '메뉴분류_해산물'],
      dtype='object')

In [757]:
X.head(2)

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,식사가능인원,중식기온,중식강수량,확진자수,중식강수여부,...,요일_화,국분류_국,국분류_국수,국분류_스프,국분류_찌개,국분류_탕,메뉴분류_기타,메뉴분류_육류,메뉴분류_조류,메뉴분류_해산물
0,-1.208083,-0.748721,-2.094550,-0.146735,-0.395901,0.202544,-1.594185,-0.170096,-0.283312,0.0,...,0,0,0,0,1,0,0,1,0,0
1,-1.208083,-0.748721,-1.565987,0.182349,-0.395901,0.072225,-1.736090,-0.170096,-0.283312,0.0,...,1,0,0,0,1,0,0,0,0,1


In [758]:
X_train, X_test, y_train = X, df_test_lunch, y

In [759]:
X_train.columns

Index(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '식사가능인원',
       '중식기온', '중식강수량', '확진자수', '중식강수여부', '휴일전날', '휴일다음날', '월_1', '월_2', '월_3',
       '월_4', '월_5', '월_6', '월_7', '월_8', '월_9', '월_10', '월_11', '월_12',
       '요일_금', '요일_목', '요일_수', '요일_월', '요일_화', '국분류_국', '국분류_국수', '국분류_스프',
       '국분류_찌개', '국분류_탕', '메뉴분류_기타', '메뉴분류_육류', '메뉴분류_조류', '메뉴분류_해산물'],
      dtype='object')

In [760]:
X_test.columns

Index(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '식사가능인원',
       '중식기온', '중식강수량', '확진자수', '중식강수여부', '휴일전날', '휴일다음날', '월_1', '월_2', '월_3',
       '월_4', '월_5', '월_6', '월_7', '월_8', '월_9', '월_10', '월_11', '월_12',
       '요일_금', '요일_목', '요일_수', '요일_월', '요일_화', '국분류_국', '국분류_국수', '국분류_스프',
       '국분류_찌개', '국분류_탕', '메뉴분류_기타', '메뉴분류_육류', '메뉴분류_조류', '메뉴분류_해산물'],
      dtype='object')

### 석식

In [761]:
df_test_dinner = df_test_dinner[['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '식사가능인원',
       '석식기온', '석식강수량', '확진자수', '석식강수여부', '휴일전날', '휴일다음날', '월_1', '월_2', '월_3',
       '월_4', '월_5', '월_6', '월_7', '월_8', '월_9', '월_10', '월_11', '월_12',
       '요일_금', '요일_목', '요일_수', '요일_월', '요일_화', '국분류_국', '국분류_국수', '국분류_스프',
       '국분류_죽', '국분류_찌개', '국분류_탕', '메뉴분류_기타', '메뉴분류_육류', '메뉴분류_조류',
       '메뉴분류_해산물']]

In [762]:
X2 = df_dinner.drop(columns=['일자','중식계','석식계','석식결식','중식결식', '중식기온', '중식강수량', '중식강수여부'])

# 석식에 경우 석식강수량과 강수여부가 휴일전날 pvaule 값이 유의미하지 않아 뺏다

#X = df[['요일','본사정원수','본사휴가자수','본사출장자수','본사시간외근무명령서승인건수','현본사소속재택근무자수','식사가능인원']] #0.43
y2 = df_dinner['석식계']



In [763]:
X2_train, X2_test, y2_train = X2, df_test_dinner, y2

In [764]:
X2.columns

Index(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '식사가능인원',
       '석식기온', '석식강수량', '확진자수', '석식강수여부', '휴일전날', '휴일다음날', '월_1', '월_2', '월_3',
       '월_4', '월_5', '월_6', '월_7', '월_8', '월_9', '월_10', '월_11', '월_12',
       '요일_금', '요일_목', '요일_수', '요일_월', '요일_화', '국분류_국', '국분류_국수', '국분류_스프',
       '국분류_죽', '국분류_찌개', '국분류_탕', '메뉴분류_기타', '메뉴분류_육류', '메뉴분류_조류',
       '메뉴분류_해산물'],
      dtype='object')

# 모델링

## 중식

In [765]:
import statsmodels.api as sm
X=sm.add_constant(X)
lr = sm.OLS(y,X)
result= lr.fit() 
print(result.summary())
print(result.params)
#재택근무자수 무의미함 

                            OLS Regression Results                            
Dep. Variable:                    중식계   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     176.8
Date:                Mon, 04 Oct 2021   Prob (F-statistic):               0.00
Time:                        23:30:18   Log-Likelihood:                -7072.0
No. Observations:                1205   AIC:                         1.421e+04
Df Residuals:                    1171   BIC:                         1.439e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            519.7314      7.421     70.

## 석식

In [766]:
lr = sm.OLS(y2,X2)
result= lr.fit() 
print(result.summary())
print(result.params)
# 저녁은 확진자수, 강수, 강수여부

                            OLS Regression Results                            
Dep. Variable:                    석식계   R-squared:                       0.639
Model:                            OLS   Adj. R-squared:                  0.629
Method:                 Least Squares   F-statistic:                     58.74
Date:                Mon, 04 Oct 2021   Prob (F-statistic):          1.76e-222
Time:                        23:30:19   Log-Likelihood:                -6520.5
No. Observations:                1161   AIC:                         1.311e+04
Df Residuals:                    1126   BIC:                         1.329e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
본사정원수            -33.1629      1.733    -19.

## 릿지

### 점심

In [767]:
from sklearn.linear_model import Ridge

param_grid = {'alpha':[0.001, 0.01, 0.1, 1,10,50]}
gird_search = GridSearchCV(Ridge(),param_grid, cv=5, scoring='r2')

gird_search.fit(X_train,y_train)

ridge_lunch_pre =gird_search.predict(X_test)

### 저녁

In [770]:
from sklearn.linear_model import Ridge

aram_grid = {'alpha':[0.001, 0.01, 0.1, 1,10,50]}
gird_search = GridSearchCV(Ridge(),param_grid, cv=5, scoring='neg_mean_absolute_error')

gird_search.fit(X2_train,y2_train)
ridge_dinner_pre =gird_search.predict(X2_test)


## 라쏘

### 점심

In [771]:
from sklearn.linear_model import Lasso #라쏘


param_grid = {'alpha':[0.001, 0.01, 0.1, 1,10,50]}
gird_search = GridSearchCV(Lasso(),param_grid, cv=5, scoring='neg_mean_absolute_error')

gird_search.fit(X_train,y_train)
lasso_lunch_pre =gird_search.predict(X_test)


### 저녁

In [772]:

param_grid = {'alpha':[0.001, 0.01, 0.1, 1,10,50]}
gird_search = GridSearchCV(Lasso(),param_grid, cv=5, scoring='neg_mean_absolute_error')

gird_search.fit(X2_train,y2_train)
lasso_dinner_pre =gird_search.predict(X2_test)


## xgboost

In [773]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBRegressor

### 점식

In [ ]:
'''param = {
    'max_depth':[2,3,4],
    'n_estimators':range(300,600,100), #  'n_estimators':range(600,700,50) 여기에 cv 10 (이거와 별반차이가 없다.)
    'colsample_bytree':[0.5,0.7,1],
    'colsample_bylevel':[0.5,0.7,1],
}

model = xgb.XGBRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param, cv=10, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

grid_search.fit(X_train, y_train)

pre = grid_search.predict(X_test)

print(grid_search.score(X_test,y_test))
print('RMSE=',np.sqrt(mean_squared_error(y_test, pre)))
print('MAE=',mean_absolute_error(y_test, pre))

with open('model_rf/XG_lunch_estimator.pickle','wb') as fw: #저장
    pickle.dump(grid_search.best_estimator_, fw)
'''

"param = {\n    'max_depth':[2,3,4],\n    'n_estimators':range(300,600,100), #  'n_estimators':range(600,700,50) 여기에 cv 10 (이거와 별반차이가 없다.)\n    'colsample_bytree':[0.5,0.7,1],\n    'colsample_bylevel':[0.5,0.7,1],\n}\n\nmodel = xgb.XGBRegressor()\ngrid_search = GridSearchCV(estimator=model, param_grid=param, cv=10, \n                           scoring='neg_mean_squared_error',\n                           n_jobs=-1)\n\ngrid_search.fit(X_train, y_train)\n\npre = grid_search.predict(X_test)\n\nprint(grid_search.score(X_test,y_test))\nprint('RMSE=',np.sqrt(mean_squared_error(y_test, pre)))\nprint('MAE=',mean_absolute_error(y_test, pre))\n\nwith open('model_rf/XG_lunch_estimator.pickle','wb') as fw: #저장\n    pickle.dump(grid_search.best_estimator_, fw)\n"

In [774]:
with open('model_rf/XG_lunch_estimator.pickle','rb') as f: #불러오기
    XG_lunch_model = pickle.load(f)

In [775]:
XG_lunch_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=2,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=300, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
XG_lunch_model.fit(X_train,y_train)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=300, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [776]:
xg_lunch_pre = XG_lunch_model.predict(X_test)

저녁

In [ ]:
'''
param = {
    'max_depth':[2,3,4],
    'n_estimators':range(300,600,100), #  'n_estimators':range(600,700,50) 여기에 cv 10 (이거와 별반차이가 없다.)
    'colsample_bytree':[0.5,0.7,1],
    'colsample_bylevel':[0.5,0.7,1],
}

model = xgb.XGBRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param, cv=10, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

grid_search.fit(X2_train, y2_train)

pre = grid_search.predict(X2_test)

print(grid_search.score(X2_test,y2_test))
print('RMSE=',np.sqrt(mean_squared_error(y2_test, pre)))
print('MAE=',mean_absolute_error(y2_test, pre))

with open('model_rf/XG_dinner_estimator.pickle','wb') as fw: ##저장
    pickle.dump(grid_search.best_estimator_, fw)
'''

"\nparam = {\n    'max_depth':[2,3,4],\n    'n_estimators':range(300,600,100), #  'n_estimators':range(600,700,50) 여기에 cv 10 (이거와 별반차이가 없다.)\n    'colsample_bytree':[0.5,0.7,1],\n    'colsample_bylevel':[0.5,0.7,1],\n}\n\nmodel = xgb.XGBRegressor()\ngrid_search = GridSearchCV(estimator=model, param_grid=param, cv=10, \n                           scoring='neg_mean_squared_error',\n                           n_jobs=-1)\n\ngrid_search.fit(X2_train, y2_train)\n\npre = grid_search.predict(X2_test)\n\nprint(grid_search.score(X2_test,y2_test))\nprint('RMSE=',np.sqrt(mean_squared_error(y2_test, pre)))\nprint('MAE=',mean_absolute_error(y2_test, pre))\n\nwith open('model_rf/XG_dinner_estimator.pickle','wb') as fw: ##저장\n    pickle.dump(grid_search.best_estimator_, fw)\n"

In [777]:
with open('model_rf/XG_dinner_estimator.pickle','rb') as f:
    XG_dinner_model = pickle.load(f)

In [778]:
XG_dinner_model =XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=2,
             min_child_weight=1,monotone_constraints='()',
             n_estimators=300, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
XG_dinner_model.fit(X2_train,y2_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=300, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [779]:
XG_dinner_pre = XG_dinner_model.predict(X2_test)


## 랜덤포레스트 모델

## 점심 저녁 한번예

In [ ]:
'''
params = {#max_depth=10, n_estimators=50, n_jobs=-1, random_state=42
    'n_estimators': [50,100,150,300,1000],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [1,3,6,10],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 3, 5, 10]}

model1 = RandomForestRegressor(n_jobs=-1, random_state=42)
model2 = RandomForestRegressor(n_jobs=-1, random_state=42)

RF_lunch_model = GridSearchCV(model1, params, scoring='neg_mean_absolute_error')
RF_dinner_model = GridSearchCV(model2, params, scoring='neg_mean_absolute_error')

RF_lunch_model.fit(X_train, y_train)
RF_dinner_model.fit(X2_train, y2_train)

RF_lunch_best = RF_lunch_model.best_score_
RF_dinner_best = RF_dinner_model.best_score_

print('점심 RF 모델 에러값(mae) : ',RF_lunch_best)
print('저녁 RF 모델 에러값(mae) : ',RF_dinner_best)

rf_lunch_param=RF_lunch_model.best_params_
rf_lunch_est = RF_lunch_model.best_estimator_

rf_dinner_param=RF_dinner_model.best_params_
rf_lunch_est = RF_dinner_model.best_estimator_

with open('model_rf/rf_lunch_estimator.pickle','wb') as fw:
    pickle.dump(RF_lunch_model.best_estimator_, fw)
with open('model_rf/rf_dinner_estimator.pickle','wb') as fw:
    pickle.dump(RF_dinner_model.best_estimator_, fw)
# 모델값저장
'''


"\nparams = {#max_depth=10, n_estimators=50, n_jobs=-1, random_state=42\n    'n_estimators': [50,100,150,300,1000],\n    'max_features': ['auto', 'sqrt'],\n    'max_depth' : [1,3,6,10],\n    'min_samples_leaf': [1, 2, 4],\n    'min_samples_split': [2, 3, 5, 10]}\n\nmodel1 = RandomForestRegressor(n_jobs=-1, random_state=42)\nmodel2 = RandomForestRegressor(n_jobs=-1, random_state=42)\n\nRF_lunch_model = GridSearchCV(model1, params, scoring='neg_mean_absolute_error')\nRF_dinner_model = GridSearchCV(model2, params, scoring='neg_mean_absolute_error')\n\nRF_lunch_model.fit(X_train, y_train)\nRF_dinner_model.fit(X2_train, y2_train)\n\nRF_lunch_best = RF_lunch_model.best_score_\nRF_dinner_best = RF_dinner_model.best_score_\n\nprint('점심 RF 모델 에러값(mae) : ',RF_lunch_best)\nprint('저녁 RF 모델 에러값(mae) : ',RF_dinner_best)\n\nrf_lunch_param=RF_lunch_model.best_params_\nrf_lunch_est = RF_lunch_model.best_estimator_\n\nrf_dinner_param=RF_dinner_model.best_params_\nrf_lunch_est = RF_dinner_model.best_esti

In [ ]:
'''print(rf_lunch_param, rf_lunch_est, rf_dinner_param, rf_lunch_est)'''

'print(rf_lunch_param, rf_lunch_est, rf_dinner_param, rf_lunch_est)'

In [780]:
with open('model_rf/rf_lunch_estimator.pickle','rb') as f:
    RF_lunch_model = pickle.load(f)
with open('model_rf/rf_dinner_estimator.pickle','rb') as f:
    RF_dinner_model = pickle.load(f)
RF_lunch_model = RandomForestRegressor(max_depth=10, n_estimators=300, n_jobs=-1,
                      random_state=42)
RF_dinner_model = RandomForestRegressor(max_depth=10, n_estimators=300, n_jobs=-1,
                      random_state=42)
RF_lunch_model.fit(X_train,y_train)
RF_dinner_model.fit(X2_train, y2_train)


RandomForestRegressor(max_depth=10, n_estimators=300, n_jobs=-1,
                      random_state=42)

In [781]:
rf_lunch_pre = RF_lunch_model.predict(X_test)
rf_dinner_pre = RF_dinner_model.predict(X2_test)

## KNN

## 점심

In [782]:
from sklearn.neighbors import KNeighborsRegressor

grid_params = {
    'n_neighbors' : list(range(1,20)),
    'weights' : ["uniform", "distance"],
    'metric' : ['euclidean', 'manhattan', 'minkowski']
}

grid_model = GridSearchCV(KNeighborsRegressor(), grid_params, scoring='neg_mean_absolute_error')

grid_model.fit(X_train, y_train) 
knn_lunch_pre = grid_model.predict(X_test)

## 저녁

In [783]:
from sklearn.neighbors import KNeighborsRegressor

grid_params = {
    'n_neighbors' : list(range(1,20)),
    'weights' : ["uniform", "distance"],
    'metric' : ['euclidean', 'manhattan', 'minkowski']
}

grid_model = GridSearchCV(KNeighborsRegressor(), grid_params, scoring='neg_mean_absolute_error')

grid_model.fit(X2_train, y2_train) 
knn_dinner_pre = grid_model.predict(X2_test)


In [784]:
sample = pd.read_csv('data/sample_submission.csv')

In [786]:
sample['중식계'] = rf_lunch_pre
sample['석식계'] = rf_dinner_pre

In [788]:
sample.to_csv('data/rf_sample.csv',index=False)

In [789]:
sample['중식계'] = lasso_lunch_pre
sample['석식계'] = lasso_dinner_pre
sample.to_csv('data/lasso_sample.csv',index=False)

In [790]:
sample['중식계'] = knn_lunch_pre
sample['석식계'] = knn_dinner_pre
sample.to_csv('data/knn_sample.csv',index=False)

In [791]:
sample['중식계'] = ridge_lunch_pre
sample['석식계'] = ridge_dinner_pre
sample.to_csv('data/ridge_sample.csv',index=False)